In [ ]:
!pip install open-clip-torch  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
!unzip private_test_cropped.zip -d .

Archive:  private_test_cropped.zip
   creating: ./private_test_cropped/
  inflating: ./private_test_cropped/private_test_4_7_crop_1_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_1_crop_1_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_1_crop_2_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_1_crop_3_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_1_crop_4_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_2_crop_1_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_2_crop_2_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_3_crop_1_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_4_crop_1_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_4_crop_2_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_4_crop_3_traffic_sign.jpg  
  inflating: ./private_test_cropped/private_test_1_4_crop_4

In [ ]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
from open_clip import create_model_from_pretrained, get_tokenizer
from urllib.request import urlopen

In [ ]:
image_files = os.listdir('private_test_cropped')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model, preprocess = create_model_from_pretrained('hf-hub:timm/ViT-SO400M-14-SigLIP-384', device=device)
tokenizer = get_tokenizer('hf-hub:timm/ViT-SO400M-14-SigLIP-384')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

open_clip_model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

## Process images and generate embeddings

### Subtask:
Iterate through the image files in the `law_images` directory, preprocess each image, and generate its embedding using the provided hint. Store the embeddings in a list.


In [ ]:
from tqdm import tqdm

image_embeddings = []

for image_file in tqdm(image_files):
    image_path = os.path.join('private_test_cropped', image_file)
    image = Image.open(image_path).convert("RGB")
    image_preprocessed = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = model.encode_image(image_preprocessed)

    embedding = F.normalize(embedding, dim=-1)
    image_embeddings.append(embedding)

100%|██████████| 342/342 [01:18<00:00,  4.35it/s]


In [ ]:
len(image_embeddings)

342

In [ ]:
import numpy as np

image_embeddings_array = torch.cat(image_embeddings, dim=0).cpu().numpy()

np.save('private_test_embeddings.npy', image_embeddings_array)

In [ ]:
with open('private_test_embeddings.txt', 'w', encoding='utf-8') as f:
  f.write(
      '\n'.join(image_files)
  )